# Initialization

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from result_saver import SaverProvider

provider = SaverProvider()

# Initialize simulator

In [56]:
from soft_info import RepCodeIQSimulator

DEVICE = 'ibm_sherbrooke'
DISTANCE = 25
ROUNDS = 25
OTHER_DATE = '2023-10-27'
_RESETS = False
LOGICAL = 0 # NOT NEEDED FOR EXTREME IQ BCS HARDCODED 0

_is_hex = True
if DEVICE == 'ibmq_mumbai':
    _is_hex = False

# Initialize simulator
simulator = RepCodeIQSimulator(provider, DISTANCE, ROUNDS, DEVICE, _is_hex=_is_hex, _resets = _RESETS, other_date=OTHER_DATE)

# Get IQ data

In [71]:
SHOTS = int(1e5)
NOISE_LIST = [3e-2, 0.8e-2, 1e-2, 3e-2] # [two-qubit-fidelity, reset error, measurement error, idle error]
# NOISE_LIST = None
P_AMBIG = 0.1

IQ_data = simulator.generate_IQ(SHOTS, noise_list=NOISE_LIST)
# IQ_data_extreme = simulator.generate_extreme_IQ(SHOTS, P_AMBIG, noise_list=NOISE_LIST)

# Decode using predecoder

In [72]:
import pymatching
import stim
import cpp_soft_info
import numpy as np

model = simulator.stim_circ.detector_error_model(decompose_errors=False)
matching = pymatching.Matching.from_detector_error_model(model)

In [ ]:
_DETAILED = False  

result_predecoder = cpp_soft_info.decode_time_nn_predecode_grid(model, IQ_data[:], ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict,
                                           simulator.processed_scaler_dict, _detailed=_DETAILED, threshold=0.15, _ntnn_edges = not _RESETS)


print(f"num_errors: {result_predecoder.decode_result.num_errors}")
print(f"mean num removed edges: {np.mean(result_predecoder.nb_rm_edges)}")
print(f"num removed edges: {np.sum(result_predecoder.nb_rm_edges)}")

num_errors: 19
mean num removed edges: 3.8307
num removed edges: 38307


In [ ]:
for threshold in np.linspace(0, 0.4, 20):
    result_predecoder = cpp_soft_info.decode_time_nn_predecode_grid(model, IQ_data[:], ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict,
                                               simulator.processed_scaler_dict, _detailed=_DETAILED, threshold=threshold, _ntnn_edges = not _RESETS)
    print(f"threshold: {threshold}, num_errors: {result_predecoder.decode_result.num_errors}, mean num removed edges: {np.mean(result_predecoder.nb_rm_edges)}, num removed edges: {np.sum(result_predecoder.nb_rm_edges)}")

threshold: 0.0, num_errors: 319, mean num removed edges: 20.0672, num removed edges: 200672
threshold: 0.044444444444444446, num_errors: 30, mean num removed edges: 6.2269, num removed edges: 62269
threshold: 0.08888888888888889, num_errors: 23, mean num removed edges: 5.3079, num removed edges: 53079
threshold: 0.13333333333333333, num_errors: 19, mean num removed edges: 4.2015, num removed edges: 42015
threshold: 0.17777777777777778, num_errors: 19, mean num removed edges: 2.993, num removed edges: 29930
threshold: 0.22222222222222224, num_errors: 20, mean num removed edges: 2.5514, num removed edges: 25514
threshold: 0.26666666666666666, num_errors: 19, mean num removed edges: 2.2554, num removed edges: 22554
threshold: 0.3111111111111111, num_errors: 21, mean num removed edges: 1.9726, num removed edges: 19726
threshold: 0.35555555555555557, num_errors: 21, mean num removed edges: 1.6847, num removed edges: 16847
threshold: 0.4, num_errors: 21, mean num removed edges: 1.3878, num r

In [ ]:
# Soft Grid info

result = cpp_soft_info.decode_IQ_fast(model, IQ_data,
                                           ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict,
                                           simulator.processed_scaler_dict, _detailed=_DETAILED, nb_intervals=-1)

print(f"num_errors: {result.num_errors}")

num_errors: 5


In [ ]:
# Hard decoder
# p_meas = 0.02
p_meas = -1

result_informed = cpp_soft_info.decode_IQ_shots_flat_informed(model, IQ_data[:],
                                           ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict, simulator.processed_scaler_dict,
                                           p_data = -1, p_mixed = -1, p_meas = p_meas, common_measure=-1, _detailed=_DETAILED, _ntnn_edges = not _RESETS)

print(f"num_errors: {result_informed.num_errors}")

num_errors: 13
